## Generate HScript function completions

In [1]:
import re
import json
import random
import subprocess
from pprint import pprint

from hfs import HFS

In [2]:
# Obtain available functions with hbatch.
proc = subprocess.run([HFS / 'bin' / 'hbatch.exe', '-q', '-c', 'exhelp;quit'],
                      stdout=subprocess.PIPE,
                      universal_newlines=True)
funcs = set(proc.stdout.split())


# Collect exhelp helpcards.
script = ';'.join(f'exhelp {f}' for f in sorted(funcs))
script += ';quit'
proc = subprocess.run([HFS / 'bin' / 'hbatch.exe', '-q', '-c', script],
                     stdout=subprocess.PIPE,
                     universal_newlines=True)
output = proc.stdout

In [3]:
data = re.sub(r'[`"<>]', '', output).strip().split('----\n')
print(f'Obtained {len(data)} expression functions')

Obtained 464 expression functions


In [4]:
functions = {}
ill = []
for helpcard in data:
    match = re.match(r'(\w+)\s*(\w+)\s*\((.*)?\)((?:.|\n)*?)USAGE\s+\2\(((?:.|\n)*?)?\)', helpcard)
    if not match:
        ill.append(helpcard)
        continue

    summary = match[4].strip().partition('\n')[0]
    summary = ' '.join(summary.split())

    extype = match[1]
    exname = match[2]

    argtypes = [a for a in (a.strip() for a in match[3].split(',')) if a]
    argnames = [a for a in (a.strip() for a in match[5].split(',')) if a]
    functions[exname] = extype, argtypes, argnames, summary
    
    # if exname == 'chrampt':
    #     print(helpcard)

print(f'Parsed {len(functions)} functions, {len(ill)} helpcards cannot be parsed:')
for i in ill:
    first_line = i.split('\n', 1)[0]
    print(f"    {first_line}")

print('Overriding with:')
with open('expressions.cmd') as f:
    for over in f.read().split('\n'):
        print(f'    {over}')
        match = re.match(r'(\w+) (\w+)\((.*)\)', over)
        if not match:
            continue
        extype, exname, args = match.group(1, 2, 3)
        argtypes, argnames = [], []
        if args:
            argtypes, argnames = zip(*[pair.split() for pair in args.split(',')])

        summary = ''
        if exname in functions:
            summary = functions[exname][3]

        functions[exname] = extype, argtypes, argnames, summary
        
for exname, (extype, argtypes, argnames, summary) in functions.items():
    if len(argtypes) != len(argnames):
        print(exname, extype, argtypes, argnames)
        raise RuntimeError
        
functions = {k: v for k, v in sorted(functions.items())}

print(f'Parsed {len(functions)} of {len(data)} total.')

Parsed 450 functions, 14 helpcards cannot be parsed:
    float arclenD (string surface_node, float prim_num, float ustart, float ustop, float divs)
    float ch (string)
    float chrampt (string ramp_path, float position, float component_index, float time)
    float explodematrixpr (matrix mat, vector p, vector trs, string xyz, string component, string)
    float imgbounds (string, string, string, float)
    float index (string, string)
    string opstreamname (string nodepath)
    float property (string path, float default)
    float propertyf (string, float, float)
    string propertys (string, string)
    string propertysop (string, string)
    string propertysraw (string, string)
    float propertyt (string, float, float)
    string rtrim (string string, string)
Overriding with:
    float arclenD(string surface_node, float prim_num, float ustart, float ustop, float divs)
    float ch(string channel)
    float chrampt(string ramp_path, float position, float component_index, float t

In [5]:
expressions = {}

for exname, (extype, argtypes, argnames, summary) in functions.items():
    expressions[exname] = {'return': extype}
    expressions[exname]['argtypes'] = argtypes
    expressions[exname]['argnames'] = argnames
    expressions[exname]['summary'] = summary

def test():
    print(len(expressions))
    f = random.choice(list(expressions.keys()))
    print(f)
    pprint(expressions[f])
    
test()

466
ch
{'argnames': ('channel',),
 'argtypes': ('string',),
 'return': 'float',
 'summary': ''}


In [6]:
comps = {
    'scope': 'source.hscript -string',
    'completions': []
}
used_triggers = set()

for exname, exdata in expressions.items():
    argnames = exdata['argnames']
    trigger = f"{exname}({', '.join(argnames)})"

    # Completion exists.
    if trigger in used_triggers:
        continue

    # Completion not exists. Make a new one.
    used_triggers.add(trigger)

    # Build contents.
    cargs = [f'${{{i}:{arg}}}' for i, arg in enumerate(argnames, 1)]
    contents = f"{exname}({', '.join(cargs)})"

    comps['completions'].append({'trigger': trigger, 'contents': contents,
                                 'kind': 'function', 'details': exdata['summary']})


print('Generated %d completions.' % len(used_triggers))


# Write completions into a functions.sublime-completions file.
with open('expressions.sublime-completions', 'w') as f:
    json.dump(comps, f, indent=4)

Generated 466 completions.
